# MongoDB파일 Elastic Search에 넣는법

## 나처럼 헤메는사람이 더 없길 바라는 의도에서 쓴다.

### 0. 전제: MongoDB, Elastic Search, Logstash가 전부 설치되어 있을 것.

### 1. MongoDB로 저장한 DB를 읽어온다.
- Logstash의 input으로 MongoDB를 지정하기 위해 필요한 작업이다.
- Logstash로 MongoDB 그냥 안읽어와진다. 플러그인 추가로 설치하라고 나올것이다.
- (Mac 기준)bin/logstash-plugin list를 쉘에다 입력하면 현재 설치되어있는 플러그인들이 다 뜬다.
- 플러그인 중에 logstash-input-mongodb 가 있는지 확인한다. 아마 없을것이다.
- bin/logstash-plugin install logstash-input-mongodb를 쉘에다 입력한다. 알아서 플러그인을 설치할것이다.
- 플러그인을 사용할 준비가 되었다.
- Mac에서밖에 안해봐서 잘 모르겠다. 설치 안되면 https://github.com/phutchins/logstash-input-mongodb 들어가서 설명 따라하면 설치 될거다.

=> 윈도우에서도 저 명령 쓰면 되더라.

### 2. Config 파일을 생성한다.
- Logstash의 input을 MongoDB로, output을 Elastic Search로 지정하기 위한 작업이다.
- 아마 Logstash가 어떤 툴인지는 아시겠지만, 혹시 해서 설명하자면 일종의 pipe라고 보면 된다. input으로 설정된 곳에서 데이터를 받아 output으로 설정된 곳으로 넘기는 역할.
- config파일 내용이다.

        input{
            mongodb{
                uri => "mongodb://123.456.789.101:27017/desired_db"
                placeholder_db_dir => "/Somewhere/to/save"
                placeholder_db_name => "savefile_name.db"
                collection => "desired_collection_name"
                batch_size => 1000
           }
        }

        filter{
           mutate{
               rename => {"_id" => "id"}
           }
        }

        output{
            elasticsearch{
                hosts => ["localhost:9200"]
            }
        }
    - input: MongoDB를 input으로 받을 것이므로 mongodb로 지정한다.
    - uri: 당신이 가져올 db가 저장된 주소다. 가령 당신이 가져올 db 이름이 temp이고 local에서 가져오는거라면 mongodb://127.0.0.1:27017/temp가 될 것이다. 잘 모르겠으면 https://docs.mongodb.org/v3.0/reference/connection-string/ 여기 가보면 자세하게 나와있다.
    - placeholder_db_dir: Logstash딴에서 작업내용을 저장할 파일을 만드는것 같다.(뇌피셜이니 적당히 걸러들으시길 바랍니다...) 여튼 이 파일을 저장할 경로를 적어주면 된다.
    - placeholder_db_name: 위에서 말한 저 파일의 이름을 지정한다.
    - 참고로 placeholder_db_dir, placeholder_db_name 두개 없으면 Logstash에서 config 파일이 잘못됨!!! 하고 뭐라뭐라 할거시다.
    - collection: 당신이 가져올 db 내의 collection 이름을 명시하면 된다.(한개만 줄 수 있더라)
    - batch_size: batch 크기 지정. 기본값은 1이다.
    ___
    - rename: 매우 중요!!! MongoDB 내용 그대로 갖다쓰면 에러나면서 하나도 안들어간다. MongoDB에 내용 입력하면 _id라는 값이 자동으로 생성되는 걸 다들 아실 것이다. Logstash에서는 _id를 인식을 못하는지 바꿔주라고 뭐라뭐라 하면서 데이터 입력이 안된다. 저 _id를 id로 바꿔주기 위해 사용했다.(다른 이름 사용해도 될 것이다)
    ___
    - output: Elastic Search로 보낼 것이므로 elastic search로 지정한다.
    - hosts: 당신이 실행한 Elastic Search의 주소다. 기본값으로 실행시켰다면 localhost:9200으로 써주면 된다.

### 3. 실행
- shell에서 bin/logstash -f "config 파일명" 으로 명령어를 입력한다.
- 제대로 따라했다면 잘 들어갔을 거시다.
- 이유는 잘 모르겠으나 내용은 잘 들어갔는데 DEBUG -- : MONGODB 어쩌고 하는 내용이 계속 나온다. 

___

본인은 저 _id 바꿔주라는걸 못알아먹어서 엄청 삽질했다... 